# Q5

In [102]:
import numpy as np
import sympy as sp
import pandas as pd


In [103]:
x = sp.symbols('x')
func_a = x**(1/3)
func_b = (x**2)*sp.exp(-x)

f_a = sp.lambdify((x), func_a)
f_b = sp.lambdify((x), func_b)

integral_func_a = sp.integrate(func_a, x)
integral_func_b = sp.integrate(func_b, x)

print(integral_func_a, "|" , integral_func_b)
true_integral_f_a = sp.lambdify((x), integral_func_a)
true_integral_f_b = sp.lambdify((x), integral_func_b)

print(f"True Integral f_a : {true_integral_f_a(1) - true_integral_f_a(0)}")
print(f"True Integral f_b : {true_integral_f_b(1) - true_integral_f_b(0)}")

0.75*x**1.33333333333333 | (-x**2 - 2*x - 2)*exp(-x)
True Integral f_a : 0.75
True Integral f_b : 0.16060279414278833


In [104]:
trapezoidal_rule = lambda f, a, b : (f(a) + f(b)) * (b-a)/2

# f_a = lambda x: x**(1/3)
# f_b = lambda x: (x**2)*np.exp(-x)

def composite_trapezoidal_rule(
        f: callable,
        a: np.float64,
        b: np.float64,
        m: np.int64
):
    integral = 0
    panels = 2**(m)
    h = (b-a)/panels

    for panel in range(1, panels+1):
        integral += trapezoidal_rule(f, (panel-1)*h, panel*h)

    return integral

def romberg_integration(f, a, b, m_upper_limit):
    R = np.zeros((m_upper_limit, m_upper_limit))
    
    for m in range(m_upper_limit):
        R[m, 0] = composite_trapezoidal_rule(f, a, b, m)
        for j in range(1, m + 1):
            R[m, j] = ( (4**j) * R[m, j-1] - R[m-1, j-1] ) / ( 4**j - 1 )

        print(abs(R[m, m] - R[m-1, m-1]))
        if m > 0 and abs(R[m, m] - R[m-1, m-1]) < 1e-5:
            return R[:m+1, :m+1]
            
    return R


In [105]:
R_f_a = pd.DataFrame(romberg_integration(f_a, 0, 1, 50))
R_f_a 

0.5
0.1958003506560666
0.0348337892932824
0.011865505328313652
0.004541574571933826
0.0017863319246311793
0.0007073345281107812
0.0002805503278568988
0.00011132104658340136
4.417625650809942e-05
1.7531207212484468e-05
6.957249132355514e-06


,0,1,2,3,4,5,6,7,8,9,10,11
0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.646850,0.695800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.708055,0.728457,0.730634,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.733100,0.741448,0.742314,0.742500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.743230,0.746606,0.746950,0.747023,0.747041,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.747297,0.748653,0.748790,0.748819,0.748826,0.748828,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.748923,0.749465,0.749520,0.749531,0.749534,0.749535,0.749535,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.749572,0.749788,0.749809,0.749814,0.749815,0.749815,0.749815,0.749815,0.000000,0.000000,0.000000,0.000000
8,0.749830,0.749916,0.749924,0.749926,0.749927,0.749927,0.749927,0.749927,0.749927,0.000000,0.000000,0.000000
9,0.749932,0.749967,0.749970,0.749971,0.749971,0.749971,0.749971,0.749971,0.749971,0.749971,0.000000,0.000000


In [106]:
R_f_b = pd.DataFrame(romberg_integration(f_b, 0, 1, 50))
R_f_b

0.18393972058572117
0.0215380371050419
0.001791154782689569
7.728568188647422e-06


,0,1,2,3
0,0.183940,0.000000,0.000000,0.000000
1,0.167786,0.162402,0.000000,0.000000
2,0.162488,0.160722,0.160611,0.000000
3,0.161080,0.160610,0.160603,0.160603
